In [1]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import joblib  # Importando joblib para salvar o modelo
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from torch.utils.data import DataLoader, TensorDataset
from AlzheimerCNN import AlzheimerNet

# 1️⃣ Carregar e pré-processar os dados
df = pd.read_csv("D:\\Projetos\\AlzheimerEarlyDetection_v4\\alzheimers_disease_data.csv")

# Mapear Diagnosis para nova classificação
def map_diagnosis(row):
    if row['Diagnosis'] == 0:
        return 0  # Sem Demência
    elif row['MMSE'] >= 24:
        return 1  # Demência Leve
    elif row['MMSE'] >= 18:
        return 2  # Demência Moderada
    else:
        return 3  # Demência Crônica

df['Diagnosis_New'] = df.apply(map_diagnosis, axis=1)

# Separar features e target
features = df.drop(columns=['PatientID', 'Diagnosis', 'Diagnosis_New', 'DoctorInCharge'])
target = df['Diagnosis_New']

# Codificar variáveis categóricas
for col in features.select_dtypes(include=['object']).columns:
    features[col] = LabelEncoder().fit_transform(features[col])

# Normalizar os dados
scaler = StandardScaler()
features = scaler.fit_transform(features)

# Converter para tensores do PyTorch
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

# Criar o modelo
model = RandomForestClassifier(n_estimators=100, random_state=42)

# 2️⃣ Treinar o modelo
model.fit(X_train, y_train)

# 3️⃣ Avaliação
accuracy = model.score(X_test, y_test)
print(f"Acurácia do modelo: {accuracy:.2f}%")

# 4️⃣ Salvar o modelo com joblib
joblib.dump(model, 'alzheimer_csv_model.joblib')  # Salvar o modelo com joblib

# Salvar o scaler (caso seja necessário para usar os dados novos posteriormente)
joblib.dump(scaler, 'scaler.pkl')


Acurácia do modelo: 0.88%


['scaler.pkl']

In [ ]:
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.long)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.long)

train_data = TensorDataset(X_train_tensor, y_train_tensor)
test_data = TensorDataset(X_test_tensor, y_test_tensor)
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
test_loader = DataLoader(test_data, batch_size=32)

# Criar modelo
input_size = X_train.shape[1]
model = AlzheimerNet(input_size)

# 3️⃣ Treinar o modelo
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

for epoch in range(50):  # 30 épocas
    model.train()
    total_loss = 0
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    
    print(f"Época {epoch+1}, Perda: {total_loss:.4f}")

# 4️⃣ Avaliação
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for X_batch, y_batch in test_loader:
        outputs = model(X_batch)
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == y_batch).sum().item()
        total += y_batch.size(0)

print(f"Acurácia: {100 * correct / total:.2f}%")

torch.save(model.state_dict(), 'alzheimer_csv_model.pth')